# Classification of reddit user posts using graph machine learning with Stellargraph

We apply the graph machine algorithm APPNP [1] to the task of classifying reddit user posts into 41 different categories using the dataset published in [2] which can be downloaded [here](http://snap.stanford.edu/graphsage/reddit.zip).  

The following is a description of the dataset [2]:

>Reddit is a large online discussion forum where users post and comment on content in different topical
>communities. We constructed a graph dataset from Reddit posts made in the month of September, 2014. The node label in this case is the community, or “subreddit”, that a post belongs to. We sampled
>50 large communities and built a post-to-post graph, connecting posts if the same user comments
>on both. In total this dataset contains 232,965 posts with an average degree of 492. We use the first
>20 days for training and the remaining days for testing (with 30% used for validation). For features,
>we use off-the-shelf 300-dimensional GloVe CommonCrawl word vectors [3]; for each post, we
>concatenated (i) the average embedding of the post title, (ii) the average embedding of all the post’s
>comments (iii) the post’s score, and (iv) the number of comments made on the post.


We demonstrate the advantage of using graph features and the scalability of the APPNP algorithm for node classification on the reddit dataset.  We first train a MLP on the node features and then propagate this model using APPNP.  Training is only done on the node features, this approach allows model training to be completed in under a 1 minute on an 8th gen quad-core i7.


**References**

1. Predict then propagate: Graph neural networks meet personalized pagerank. J. Klicpera,  A. Bojchevski, & S. Günnemann arxiv:1810.05997, 2018.


2. Inductive Representation Learning on Large Graphs. W.L. Hamilton, R. Ying, and J. Leskovec arXiv:1706.02216 [cs.SI], 2017.


3. Glove: Global vectors for word representation. J. Pennington, R. Socher, and C. D. Manning. In EMNLP, 2014.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import os

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegressionCV

import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator, FullBatchNodeGenerator, SparseFullBatchNodeSequence
from stellargraph.layer import GraphSAGE, GCN, GAT, APPNP
from stellargraph.layer.appnp import APPNPPropagationLayer
from stellargraph import globalvar
from stellargraph.core.utils import GCN_Aadj_feats_op

from tensorflow.keras import layers, optimizers, losses, metrics, Model, models
from sklearn import preprocessing, feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics
from scipy.sparse import coo_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import json

%matplotlib inline

/Users/kieranricardo/anaconda3/envs/stellar-demos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kieranricardo/anaconda3/envs/stellar-demos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kieranricardo/anaconda3/envs/stellar-demos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

## Loading the Reddit Dataset

First, we load the reddit dataset which is stored as a series of json files.  We first load the graph data and then the node features and labels and ensure that indexing is consistent across the graph, labels, and node features.

In [2]:
data_dir = os.path.expanduser("~/data/reddit")

In [3]:
with open(os.path.join(data_dir, "reddit-G.json")) as gfile:
    graph_data = json.load(gfile)
    
list_node_ids = list(d['id'] for d in graph_data['nodes'])

edge_generator = ((link['source'], link['target']) for link in graph_data['links'])
edge_df = pd.DataFrame(edge_generator, columns=["target", "source"])

In [4]:
print("Number of nodes:", len(list_node_ids))
print("Number of edges:", len(edge_df))

Number of nodes: 231443
Number of edges: 11606919


In [5]:
with open(os.path.join(data_dir, "reddit-class_map.json")) as tfile:
    labels = json.load(tfile)

feats = np.load(data_dir + "/reddit-feats.npy")

feats[:,0] = np.log(feats[:,0]+1.0)
feats[:,1] = np.log(feats[:,1]-min(np.min(feats[:,1]), -1))

feat_id_map = json.load(open(data_dir + "/reddit-id_map.json"))

# sort node features to match the order of feat_id_map
sorted_idxs = np.array([feat_id_map[key] for key in list_node_ids])
feats = feats[sorted_idxs,:]

#sort node labnels to match the order of feat_id_map
labels = np.array([labels[key] for key in list_node_ids])

In [6]:
node_data = pd.DataFrame(feats)

target_encoding = preprocessing.OneHotEncoder(sparse=False, categories='auto')
targets = target_encoding.fit_transform(labels.reshape(-1, 1))
targets = pd.DataFrame(targets)

We then split the data into train/val/test based on the labels stored in the dataset. This is the same train/val/test split used in the graphsage paper. Then we fit a standard scaler on only the training data and use it to standardize all of the data.

In [7]:
def map_node_to_split(node):
    if node['test']:
        return 'test'
    elif node['val']:
        return 'val'
    else:
        return 'train'

    
train_test_val_dict = dict(zip(list_node_ids, map(map_node_to_split, graph_data['nodes'])))

train_mask = [(train_test_val_dict[key] == 'train') for key in list_node_ids]
val_mask = [(train_test_val_dict[key] == 'val') for key in list_node_ids]
test_mask = [(train_test_val_dict[key] == 'test') for key in list_node_ids]

scaler = preprocessing.StandardScaler()
scaler.fit(node_data[train_mask].values)
node_data.iloc[:, :] = scaler.transform(node_data.values)

train_data, train_targets = node_data[train_mask], targets[train_mask]
val_data, val_targets = node_data[val_mask], targets[val_mask]
test_data, test_targets = node_data[test_mask], targets[test_mask]

print("{} training nodes.".format(len(train_data)))
print("{} validation nodes.".format(len(val_data)))
print("{} testing nodes.".format(len(test_data)))

152410 training nodes.
23699 validation nodes.
55334 testing nodes.


## Creating an adjacnecy matrix and StellarGraph generator

Standard stellargaph workflows involve creating a stellargaph object and using this to create the data generators. For the large reddit graph this requires approximately 20GB of RAM. This demo has a `low_memory` option which when set to `True` creates the data generator directly from the node features and edge list and only requires approximately 8GB of RAM.

In [8]:
low_memory = True

In [13]:
def create_adj_from_edgelist(edge_df, nodelist):
    '''
    This function creates an adjacency matrix directly from an edgelist and a nodelist.
    '''
    
    node_index = dict(zip(nodelist, range(len(nodelist))))
    
    rows = [node_index[source_node] for source_node in edge_df["source"]]
    cols = [node_index[target_node] for target_node in edge_df["target"]]
    data = np.ones(len(edge_df), np.float64)
    
    Aadj = coo_matrix(
                (data, (rows, cols)),
                shape=(len(nodelist), len(nodelist)),
            )
    
    return Aadj


def create_generator_from_edge_df_feats(edge_df, node_data):
    '''
    This function is a hack that creates stellargaph FullBatchNodeGenerator directly
    from an adjacency list and a data frame of node features.
    '''
    
    # create adjacency matrix
    
    Aadj = create_adj_from_edgelist(edge_df, node_data.index)

    _, Aadj = GCN_Aadj_feats_op(
                    features=node_data, A=Aadj, method='gcn'
        )
    
    # create a dummy StellarGraph object
    gnx = nx.Graph()
    gnx.add_nodes_from(["a", "b"])

    gnx.add_edges_from([("a", "b")])

    gnx = gnx.to_undirected()
    features = np.array([[1, 1], [1, 0]])

    nodes = gnx.nodes()
    features = pd.DataFrame.from_dict(
        {n: f for n, f in zip(nodes, features)}, orient="index"
    )

    G = sg.StellarGraph(gnx, node_type_name="node", node_features=features)
    
    # create a FullBatchNodeGenerator from the dummy StellarGraph object
    generator = FullBatchNodeGenerator(G, method="gcn", sparse=True)
    
    # manually set FullBatchNodeGenerator attributes
    generator.features = node_data
    generator.Aadj = Aadj
    generator.node_list = list(node_data.index.values)
    
    return generator

In [14]:
if low_memory == True:
    
    generator = create_generator_from_edge_df_feats(edge_df, node_data)

else:
    gnx = nx.from_pandas_edgelist(edgelist)
    G = sg.StellarGraph(gnx, node_features=node_data)

    gnx.clear()
    generator = FullBatchNodeGenerator(G, method="gcn", sparse=True)

test_gen = generator.flow(test_data.index, test_targets)

Using GCN (local pooling) filters...
Using GCN (local pooling) filters...


## Training

We now create a MLP and train on the node features.

In [15]:
in_layer = layers.Input(shape=(train_data.shape[-1]))

layer = layers.Dense(512, activation='relu', kernel_regularizer="l2")(in_layer)
layer = layers.Dropout(0.5)(layer)
layer = layers.Dense(512, activation='relu', kernel_regularizer="l2")(in_layer)
layer = layers.Dropout(0.5)(layer)

#note the dimension of the output should equal the number of classes to predict!
layer = layers.Dense(train_targets.shape[-1], activation='softmax')(layer)

fully_connected_model = Model(inputs=in_layer, outputs=layer)

fully_connected_model.compile(
    optimizer=optimizers.Adam(lr=0.0001),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
history = fully_connected_model.fit(
    train_data, train_targets,
    epochs = 60,
    validation_data = (val_data, val_targets),
    batch_size = 300,
    verbose = 0
)

The MLP attains an accuracy of ~70% on the test set.

In [17]:
test_metrics = fully_connected_model.evaluate(test_data, test_targets)
print("\nTest Set Metrics:")
for name, val in zip(fully_connected_model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

55334/55334 [==============================] - 2s 27us/sample - loss: 1.1751 - acc: 0.7209

Test Set Metrics:
	loss: 1.1751
	acc: 0.7209


## APPNP Propagation

We now create an APPNP model and propagate the MLP. No further training is happening in this step.  We then test the propagated model on the test data.

In [18]:
appnp = APPNP(layer_sizes=[train_targets.shape[-1]], 
              activations=['relu'], 
              bias=True,
              generator=generator, 
              teleport_probability=0.2, 
              dropout=0.5, 
              kernel_regularizer='l2'
)

x_inp, x_out = appnp.propagate_model(fully_connected_model)
predictions = layers.Softmax()(x_out)

propagated_model = Model(inputs=x_inp, outputs=predictions)
propagated_model.compile(loss='categorical_crossentropy', metrics=['acc'],
                  optimizer='Adam')

In [19]:
test_metrics = propagated_model.evaluate_generator(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(propagated_model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))


Test Set Metrics:
	loss: 3.5071
	acc: 0.8952


Propagating the MLP with APPNP increases the test set accuracy by ~15% without any further training. As we are performing single-label multiclass classification the accuracy is equivalent to the micro F1 metric. This micro F1 is comparable to that attained in the GraphSAGE paper [2]. GraphSAGE with LSTM aggregation attains a best supervised F1 of 0.95 in [2], however APPNP only required ~3 minutes of training on an 8th gen i7 compared to the hours required for GraphSAGE while still attaining a similar F1.